# Packages

In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Open Nighttime Lights\\03_Basic Operations on Raster Files'

# Clip a VIIRS-DNB Monthly composite

Satellite data, which often comes structured as a GeoTIFF can cover large areas geospatially. It’s not always required to work with the entire file, in fact, it’s often preferred to work only with a smaller Area Of Interest (AOI).

Let's look at a VIIRS-DNB monthly composite from Decemebr 2019.

In [2]:
# Get December 2019 image, using the `avg_rad` band
viirs2019_12 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
        .filterDate('2019-12-01', '2019-12-31')\
        .select('avg_rad')\
        .median()

# Initialize map
map1 = geemap.Map()
map1.add_basemap('SATELLITE')
map1.addLayer(viirs2019_12, {}, "VIIRS-DNB Dec 2019")

map1.addLayerControl()
map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Clip to an area around Los Angeles, CA

In [3]:
lat = 34.0469
lon = -118.2541
zoom = 7

# Create a 200,000m (200km) buffer around the area
aoi = ee.Geometry.Point([lon, lat]).buffer(200000)
aoi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.buffer",
    "arguments": {
      "distance": {
        "constantValue": 200000
      },
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Point",
          "arguments": {
            "coordinates": {
              "constantValue": [
                -118.2541,
                34.0469
              ]
            }
          }
        }
      }
    }
  }
})

In [4]:
# Clip our VIIRS image to our AOI
viirs2019_12_clipped = viirs2019_12.clip(aoi)

map2 = geemap.Map(center = [lat, lon], zoom = zoom)
map2.add_basemap('SATELLITE')
map2.addLayer(viirs2019_12_clipped, {}, 'VIIRS-DNB - Greater LA Dec 2019')
map2.addLayerControl()
map2

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## Clip an image to the State of California

We arbitrarily chose a lat/lon point and buffer to clip our image around, but in geospatial analysis, we’re often called to focus on particular geopolitical boundaries.

One example might be to use the shapefile of a country or sub-national boundary to clip our image.

Conveniently, Google Earth Engine actually has geometries for US States as a native FeatureCollection so we dont need to get those. These are based on the US Census TIGER files from 2016 and are located in GEE at `TIGER/2016/States`

In [5]:
aoi_CA = ee.FeatureCollection('TIGER/2016/States')\
        .filter(ee.Filter.eq('NAME', 'California'))
aoi_CA

Name,Type,Description
ALAND,DOUBLE,Land area
AWATER,DOUBLE,Water area
DIVISION,STRING,Division code
FUNCSTAT,STRING,Functional Status
GEOID,STRING,State identifier; state FIPS code
INTPTLAT,STRING,Internal point latitude
INTPTLON,STRING,Internal point longitude
LSAD,STRING,Legal/statistical area description for state
MTFCC,STRING,MAF/TIGER feature class code (=G4000)
NAME,STRING,State name


In [6]:
# Clip VIIRS Dec 2019 to California
viirs2019_12_ca = viirs2019_12.clip(aoi_CA)

map3 = geemap.Map(center = [lat, lon], zoom = 6)
map3.add_basemap('SATELLITE')
map3.addLayer(viirs2019_12_ca, {}, 'VIIRS-DNB - California Dec 2019')
map3.addLayerControl()
map3

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## Clip an entire ImageCollection to California

So far, we’ve just clipped the December 2019 VIIRS-DNB composite, but we can easily apply this geometry clipping to an entire Image Collection. This would help if we wanted to create time series analysis of our focus area.

Previously, we passed our geometry (e.g. `aoi_CA`) to the `.clip()` function to clip a single image. We can map a function that does this to our entire ImageCollection in Python by first defining a simple function that clips a single image to our specific AOI (California).

In [7]:
# Get the entire VIIRS-DNB selection, still only selecting the `avg_rad` band
viirsDNB = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
        .select('avg_rad')

def clip_func(x, aoi = aoi_CA):
    return x.clip(aoi)

# Conversely, we can use lambda function
clip_func = lambda x: x.clip(aoi_CA)

viirs_CA = viirsDNB.map(lambda x: x.clip(aoi_CA))
viirs_CA

Name,Description
avg_rad,Average DNB radiance values.
cf_cvg,Cloud-free coverages; the total number of observations that went into each pixel. This band can be used to identify areas with low numbers of observations where the quality is reduced.


In [8]:
# Look at June 2015 and June 2019
viirs_CA_2015_06 = viirs_CA.filterDate('2015-06-01', '2015-06-30').median()
viirs_CA_2019_06 = viirs_CA.filterDate('2019-06-01', '2019-06-30').median()

map4 = geemap.Map(center = [lat, lon], zoom = 5)
map4.add_basemap('SATELLITE')
map4.addLayer(viirs_CA_2015_06, {}, 'VIIRS-DNB - California June 2015')
map4.addLayer(viirs_CA_2019_06, {}, 'VIIRS-DNB - California June 2019')
map4.addLayerControl()
map4

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [9]:
# Make an annual composite for all of 2 by reducing the ImageCollection via .median()
viirs_CA_2019 = viirs_CA.filterDate('2019-01-01', '2019-12-31').median()

map5 = geemap.Map(center = [lat, lon], zoom = 5)
map5.add_basemap('SATELLITE')
map5.addLayer(viirs_CA_2019, {}, 'VIIRS-DNB - Californioa 2019 (median)')
map5.addLayerControl()
map5

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…